## Libraries Involved

In [19]:
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R
import open3d as o3d
import time
import os
import json
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from data_repository.lookup_table import initialise_lookup_table, add_row, edit_row, delete_row, read_table
# from segmentimage import call_yolo
# import data_repository.lookup_table as lookup

## Helper Functions

In [20]:
def load_csv(file_path):
    try:
        df = pd.read_csv(file_path, sep=";")
        print(df.shape)
        if df.empty:
            print("The CSV file is empty.")
            return None
        return df
    except pd.errors.EmptyDataError:
        print("No data in the CSV file.")
        return None
    except pd.errors.ParserError:
        print("Error parsing the CSV file.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [21]:
def get_loaded_camera_matrices(camera_path):
    with open(camera_path, 'r') as f:
        loaded_data = json.load(f)
        K_loaded = np.array(loaded_data['K'])
        RT_loaded = np.array(loaded_data['RT'])
        resx = int(loaded_data['resolution_x'])
        resy = int(loaded_data['resolution_y'])

    return resx, resy, K_loaded, RT_loaded

In [22]:
def compute_projection_matrix(icm, ecm):

    R = ecm[:3, :3] 
    t = ecm[:3, 3].reshape(-1, 1)

    # Compute projection matrix P = K * [R | t]
    P = np.dot(icm, np.hstack((R, t)))

    return P

In [23]:
def show_projection(image_path, p2d):
    image = Image.open(image_path)

    image_np = np.array(image)

    points = np.array(p2d)

    x = points[:, 0]
    y = points[:, 1]

    plt.imshow(image_np)

    plt.scatter(x, y, color='blue', marker='o', alpha=0.1)

    plt.title('Image with Points')
    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')

    plt.show()

In [24]:
def hex_to_rgb(hex_code):
  if not hex_code.startswith("#") or len(hex_code) != 7:
    raise ValueError("Invalid hex code format.")

  rgb_code = hex_code[1:]
  try:
    r, g, b = tuple(int(rgb_code[i:i+2], 16) for i in (0, 2, 4))
  except ValueError:
    raise ValueError("Invalid hex code format.")

  return r, g, b

In [25]:
def get_point_cloud(file_path, noise = False):
    df = load_csv(file_path)
    if not noise: coord = ['X', 'Y', 'Z','red', 'green', 'blue']
    else: coord = ['X_noise', 'Y_noise', 'Z_noise','red', 'green', 'blue']

    return df[coord]

In [26]:
async def vis(resx, resy, icm, ecm, file_path, noise):
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name='Virtual Information Space', width=950, height=1080, left=0, top=35)
    vis.add_geometry(pcd)
    camera_lines = o3d.geometry.LineSet.create_camera_visualization(view_width_px=resx, view_height_px=resy, intrinsic=icm, extrinsic=ecm)
    vis.add_geometry(camera_lines)

    last_modified_time = os.path.getmtime(file_path)
    print(time.ctime(last_modified_time))

    try:
        while True:
            current_modified_time = os.path.getmtime(file_path)
            if current_modified_time != last_modified_time:
                temp = get_point_cloud(file_path, noise)
                if temp is not None:
                    pcd = temp
                    print(pcd.points)
                    vis.clear_geometries()
                    vis.add_geometry(pcd)
                    vis.add_geometry(camera_lines)
                last_modified_time = current_modified_time
            
            vis.update_geometry(pcd)
            vis.poll_events()
            vis.update_renderer()

    except KeyboardInterrupt:
        print("Stopped by user")
        
    vis.destroy_window()

In [27]:
def voxel_grid_downsampling(df, voxel_size=1):
    """ 
    Perform voxel grid downsampling on a DataFrame.

    Parameters:
    - df: pandas DataFrame
        Input DataFrame with columns x, y, z, and classification.
    - voxel_size: int, optional (default=1)
        Size of the voxel for downsampling.

    Returns:
    - downsampled_df: pandas DataFrame
        DataFrame after downsampling, retaining the classification.
    """
    
    df_copy = df.copy()

    df_copy['x_voxel'] = df_copy['X'] // voxel_size * voxel_size
    df_copy['y_voxel'] = df_copy['Y'] // voxel_size * voxel_size
    df_copy['z_voxel'] = df_copy['Z'] // voxel_size * voxel_size

    downsampled_df = df_copy.groupby(['x_voxel', 'y_voxel', 'z_voxel'], as_index=False).first()

    downsampled_df.drop(['x_voxel', 'y_voxel', 'z_voxel'], axis=1, inplace=True)

    return downsampled_df

In [28]:
def visualize(df:pd.DataFrame, resx, resy, icm, ecm):
    """ 
    It opens x,y,z points using open3d

    parameters:
    - df:pd.DataFrame - it should only contain x, y, z columns
    """

    points = np.vstack([df['X'], df['Y'], df['Z']]).T

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(df[['r','g','b']].values)

    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name='Virtual Information Space', width=950, height=1080, left=0, top=35)
    vis.add_geometry(pcd)
    camera_lines = o3d.geometry.LineSet.create_camera_visualization(view_width_px=resx, view_height_px=resy, intrinsic=icm, extrinsic=ecm)
    vis.add_geometry(camera_lines)

    vis.run()
    vis.destroy_window()

In [29]:
def visualize_without_colors(df:pd.DataFrame, resx, resy, icm, ecm):
    """ 
    It opens x,y,z points using open3d

    parameters:
    - df:pd.DataFrame - it should only contain x, y, z columns
    """

    points = np.vstack([df['X'], df['Y'], df['Z']]).T

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    # pcd.colors = o3d.utility.Vector3dVector(df[['r','g','b']].values)

    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name='Virtual Information Space', width=950, height=1080, left=0, top=35)
    vis.add_geometry(pcd)
    camera_lines = o3d.geometry.LineSet.create_camera_visualization(view_width_px=resx, view_height_px=resy, intrinsic=icm, extrinsic=ecm)
    vis.add_geometry(camera_lines)

    vis.run()
    vis.destroy_window()

In [30]:
object_map = {
    (255, 0, 226): 'wall',
    (0, 2, 61): 'table',
    (0, 255, 222): 'sofa',
    (63, 72, 204): 'sofa pillow1',
    (0, 162, 232): 'sofa pillow2',
    (163, 73, 164): 'sofa pillow3',
    (255, 0, 93): 'floor mat',
    (231, 255, 0): 'kitchen furniture',
    (0, 255, 22): 'vase',
    (255, 127, 39): 'desk',
    (255, 186, 0): 'tv',
    (0, 0, 0) : 'unclassified'
}

In [31]:
object_map = {
    (255, 0, 128) : (1, 1,'toilet'),
    (255, 0, 96) : (2, 1, 'toilet lid'),
    (96, 0, 255) : (3, 1, 'sofa frame'),
    (159, 0, 255) : (4, 1, 'sofa cushion'),
    (255, 0, 159) : (5, 1, 'table top'), 
    (255, 0, 191) : (6, 1, 'table base'),
    (0, 0, 0) : (0, 0, 'unclassified')
}

memoryrep = {}

for x in object_map.values():
    memoryrep[(x[0],x[1])] = []
memoryrep

{(1, 1): [],
 (2, 1): [],
 (3, 1): [],
 (4, 1): [],
 (5, 1): [],
 (6, 1): [],
 (0, 0): []}

## Model Compilation

In [32]:
def projection_model(file_path, image_path, object_map, camera_path, noise = False):
    pcd = get_point_cloud(file_path, noise)
    resx, resy, icm, ecm = get_loaded_camera_matrices(camera_path)
    P = compute_projection_matrix(icm, ecm)

    img = Image.open(image_path)
    pixels = img.load()
    width, height = img.size

    Points2DLabel = []
    PointsColors = []

    points_np = pcd[['X','Y','Z']].values

    points_homogeneous = np.hstack((points_np, np.ones((points_np.shape[0], 1))))
    points_2D_homogeneous = np.dot(P, points_homogeneous.T).T
    points_2D = points_2D_homogeneous / points_2D_homogeneous[:, 2][:, np.newaxis]

    for i in tqdm(range(len(points_2D))):
        xc, yc, _ = points_2D[i]
        if 0 <= xc < width and 0 <= yc < height:
            r, g, b = pixels[int(xc), int(yc)]
            color = (r, g, b)
            if color in object_map:
                Points2DLabel.append(list(object_map[color]))
                PointsColors.append([r, g, b])
            else:
                Points2DLabel.append([0, 0, 'unclassified'])
                PointsColors.append([0,0,0])

    return points_np, PointsColors, Points2DLabel, resx, resy, icm, ecm

In [33]:
src_path = 'C:\\Programming\\Fourth Semester\\PointVIS\\PointVIS\\assets\\output\\'

for id in range(1, 4):

    file_path = src_path + f'view{id}.csv'
    camera_path = src_path + f'camera{id}.csv'
    image_path = src_path + f'view{id}_frame_17_image_segmented.png'

    points_np, PointsColors, Points2DLabel, resx, resy, icm, ecm = projection_model(file_path, 
                                                                    image_path, 
                                                                    object_map, 
                                                                    camera_path, 
                                                                    noise = False
                                                                    )

    table = pd.concat([pd.DataFrame(points_np), pd.DataFrame(PointsColors), pd.DataFrame(Points2DLabel)], axis=1)
    table.columns = columns=['X','Y','Z','r','g','b', 'objectID', 'instanceID', 'class_name']
    summary = table[['objectID', 'instanceID', 'X','Y','Z']].groupby(['objectID', 'instanceID']).aggregate('mean')
    tablegrp = table.groupby(['objectID', 'instanceID'])
    # visualize(table, resx, resy, icm, ecm)

    path = 'data_repository/'
    for name, group in tablegrp:
        ipath = path+f'{name[0]}.{name[1]}'

        cX, cY, cZ = summary.loc[name]['X'], summary.loc[name]['Y'], summary.loc[name]['Z']

        look_up = read_table()

        for row in look_up.values: 
            if name[0] == row[0]: 
                cXO, cYO, cZO = row[3:6]
                object1 = pd.read_csv(row[len(row) - 1])

                icd = np.sqrt((cX - cXO)**2 + 
                                (cY - cYO)**2 + 
                                (cZ - cZO)**2
                                )
                
                rad = np.max(np.sqrt((object1['X'] - cXO)**2 + 
                                (object1['Y'] - cYO)**2 + 
                                (object1['Z'] - cZO)**2)
                        )
                
                if icd < rad:
                    merged = pd.concat([object1, group[['X', 'Y', 'Z']]],axis=0)
                    merged = voxel_grid_downsampling(merged, voxel_size=0.01)
                    ipath = path+f'{row[0]}.{row[1]}'
                    print(f"merging {group['class_name'].unique()[0]} and {row[2]}")
                    # visualize(merged, resx, resy, icm, ecm)
                    merged[['X','Y','Z']].to_csv(ipath)

                    points = merged[['X', 'Y', 'Z']].values

                    pcd = o3d.geometry.PointCloud()
                    pcd.points = o3d.utility.Vector3dVector(points)

                    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

                    aabb = pcd.get_axis_aligned_bounding_box()

                    obb = pcd.get_oriented_bounding_box()

                    centroid = pcd.get_center()

                    covariance = np.cov(points.T)
                    eigenvalues, eigenvectors = np.linalg.eig(covariance)

                    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)

                    surface_area = mesh.get_surface_area()

                    is_watertight = mesh.is_watertight()
                    volume = mesh.get_volume() if is_watertight else -1

                    hull, _ = pcd.compute_convex_hull()
                    # hull_volume = hull.get_volume()
                    hull_surface_area = hull.get_surface_area()

                    diameter = pcd.get_max_bound() - pcd.get_min_bound()

                    edit_row(name[0],name[1],group['class_name'].unique()[0], cX, cY, cZ, ipath, aabb, obb, eigenvalues, [], -1, hull_surface_area, diameter, volume, surface_area)
                    break
        else:
            group = voxel_grid_downsampling(group, voxel_size=0.01)
            # visualize(group, resx, resy, icm, ecm)
            group[['X','Y','Z']].to_csv(ipath)

            points = group[['X', 'Y', 'Z']].values

            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(points)

            pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

            aabb = pcd.get_axis_aligned_bounding_box()

            obb = pcd.get_oriented_bounding_box()

            centroid = pcd.get_center()

            covariance = np.cov(points.T)
            eigenvalues, eigenvectors = np.linalg.eig(covariance)

            mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)

            surface_area = mesh.get_surface_area()

            is_watertight = mesh.is_watertight()
            volume = mesh.get_volume() if is_watertight else -1

            hull, _ = pcd.compute_convex_hull()
            # hull_volume = hull.get_volume()
            hull_surface_area = hull.get_surface_area()

            diameter = pcd.get_max_bound() - pcd.get_min_bound()

            add_row(name[0],name[1],group['class_name'].unique()[0], cX, cY, cZ, ipath, aabb, obb, eigenvalues, [], -1, hull_surface_area, diameter, volume, surface_area)
            
    # objects = []
    # look_up = read_table()
    # path = 'data_repository/'
    # for name in look_up[['objectID', 'instanceID']].values:
    #     # visualize(group, resx, resy, icm, ecm)
    #     ipath = path+f'{name[0]}.{name[1]}'
    #     objects.append(pd.read_csv(ipath))

    # vis = pd.concat(objects, axis=0)

    # visualize_without_colors(vis, resx, resy, icm, ecm)


(30397, 14)


100%|██████████| 30397/30397 [00:00<00:00, 620999.29it/s]


Row added successfully.
Row added successfully.
Row added successfully.
Row added successfully.
Row added successfully.
Row added successfully.
Row added successfully.
(34690, 14)


100%|██████████| 34690/34690 [00:00<00:00, 284187.14it/s]


merging unclassified and unclassified
Row updated successfully.
merging toilet and toilet
Row updated successfully.
merging toilet lid and toilet lid
Row updated successfully.
merging sofa frame and sofa frame
Row updated successfully.
merging sofa cushion and sofa cushion
Row updated successfully.
merging table top and table top
Row updated successfully.
merging table base and table base
Row updated successfully.
(37892, 14)


100%|██████████| 37892/37892 [00:00<00:00, 336536.91it/s]


merging unclassified and unclassified
Row updated successfully.
merging toilet and toilet
Row updated successfully.
merging toilet lid and toilet lid
Row updated successfully.
merging sofa frame and sofa frame
Row updated successfully.
merging sofa cushion and sofa cushion
Row updated successfully.
merging table top and table top
Row updated successfully.
merging table base and table base
Row updated successfully.
